<h1> Activate chs_env virtual enviroment </h1>
Set-ExecutionPolicy -Scope CurrentUser -ExecutionPolicy Unrestricted <br>
chess_env\Scripts\activate

In [7]:
# Importing all relevant and necessary modules
import numpy as np 
import pandas as pd 
import os
import chess
import math
import matplotlib.pyplot as plt
import ollama
import random


In [8]:
def gen_legal_moves(board,frac=1):
    legal_moves = board.legal_moves
    legal_moves = [str(move) for move in legal_moves]
    if frac == 1:
        return legal_moves
    else:
        return np.random.choice(legal_moves, math.floor(frac*len(legal_moves)), replace=False)

def gen_prompt2(board,moves, n=10):
    return f"You are playing chess and the board state is {board}. Here you are given a list of legal moves {random.shuffle(moves)}. You have to chose the best move to make next from the list of legal moves, please provide an explanation as to why you chose this move and also provide it in UCI notation?"
    
def gen_legal_moves(board,frac=1):
    legal_moves = board.legal_moves
    legal_moves = [str(move) for move in legal_moves]
    if frac == 1:
        return legal_moves
    else:
        return np.random.choice(legal_moves, math.floor(frac*len(legal_moves)), replace=False)

def prompt_move(prompt,board):
    response = ollama.chat(model='llama2', messages=[
            {'role': 'user','content': prompt,},])
    move = response['message']['content']
    for k in range(len(move)):
        if move[k:k+4] in gen_legal_moves(board):
            move = move[k:k+4]
            break
    return move

In [3]:
df = pd.read_csv(r"chessData.csv")

In [9]:
def prompt_ensemble(size,fen):
    moves = {}  #Dictionary to store the moves and their frequencies
    for i in range(size):
        board = chess.Board(fen)
        move = prompt_move(gen_prompt2(board,gen_legal_moves(board)),board)
        if move in moves:
            moves[move] += 1 #Increment the frequency of the move
        else:
            moves[move] = 1 #Add the move to the dictionary
    return moves

def ensemble_manager(df,sizes):
    ensemble = {}
    for size in sizes:
        ensemble[size] = prompt_ensemble(size,df['FEN'][0]) #df['FEN'][0] is the first FEN in the dataset change for iteration over all FENs

    return ensemble

In [10]:
fen = df['FEN'][0]
size = 5
ensemble = prompt_ensemble(size,fen)
ensemble

{"\nAfter examining the board state you provided, I analyzed the available moves for both players. Based on the position on the board, I recommend playing the pawn at e5. Here is my reasoning:\n\n1. The pawn at e5 is well-supported by other pawns on either side of it, which will make it more difficult for Black to push it forward.\n2. The king's bishop at b5 is somewhat weakened due to the presence of Black's knight at k7. Playing the pawn at e5 will help to improve the bishop's development and potential attacking possibilities.\n3. Playing the pawn at e5 also prepares for a possible push of the queen's pawn to e4, which could open up lines for the rook at f1 to potentially attack Black's position.\n\nHere is the move in UCI notation:\n\n1. e5\n\nExplanation: The move e5 is chosen to support the pawn structure and develop the bishop, while also preparing for a possible push of the queen's pawn to e4.": 1,
 '\nAfter analyzing the board state, I would recommend playing the move "Qf3" (UC

In [6]:
gen_legal_moves(chess.Board(fen))

['g8h6',
 'g8f6',
 'b8c6',
 'b8a6',
 'h7h6',
 'g7g6',
 'f7f6',
 'e7e6',
 'd7d6',
 'c7c6',
 'b7b6',
 'a7a6',
 'h7h5',
 'g7g5',
 'f7f5',
 'e7e5',
 'd7d5',
 'c7c5',
 'b7b5',
 'a7a5']